In [1]:
import Backbone
import requests
from io import BytesIO
from PIL import Image
from position_encoder import PositionEncoder
from Encoder import Encoder
import OA_Candidate_Sampler 
import torch.nn as nn
from transformers import CLIPTextModel, CLIPTokenizer
from importlib import reload
import json

reload(Backbone)
reload(OA_Candidate_Sampler)

c:\Users\bparaju\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'OA_Candidate_Sampler' from 'c:\\Users\\bparaju\\OneDrive - Oklahoma A and M System\\ML Project\\Bibek\\Project Code\\OA_Candidate_Sampler.py'>

In [2]:
with open('unique_OA_Pairs.json','r') as f:
    UniqeOAPairs = json.load(f)

UniqeOAPairs = [tuple(pair) for pair in UniqeOAPairs]
num_oas = len(UniqeOAPairs) 

with open('OA_Pairs.json','r') as f:
    id2OAPairs = json.load(f)

id2OAPairs = {id: list(set(tuple(pair) for pair in id2OAPairs[id])) for id in id2OAPairs.keys()}

with open('vcoco_processed_data_all_2014.json', 'r') as f:
    vcoco_data = json.load(f)
    
ids = list(vcoco_data.keys())

train_ids = ids[:8000]
test_ids = ids[8000:10000]

train_data = [vcoco_data[id] for id in train_ids]
test_data = [vcoco_data[id] for id in test_ids]



In [3]:
import torch
id2OAOneHot = {}

for id, pairs in id2OAPairs.items():
    # Filter pairs that are in UniqeOAPairs
    valid_pairs = [pair for pair in pairs if pair in UniqeOAPairs]

    # Get indices of valid pairs
    indices = [UniqeOAPairs.index(pair) for pair in valid_pairs]

    # Convert indices to one-hot encoding
    oneHot = nn.functional.one_hot(torch.tensor(indices, dtype=int), num_classes=len(UniqeOAPairs))#gives batch of hot encoding (len(indices), len(UNiqeOAPairs))
    
    oneHot = oneHot.sum(axis = 0) #converts to single vector (1, len(UNiqeOAPairs))
    # Store the one-hot encodings for the ID
    id2OAOneHot[id] = oneHot

In [4]:
import numpy as np
def get_image(file,url = False):
    if(url):
      response = requests.get(img_url)
      img = Image.open(BytesIO(response.content))
      img = img.convert("RGB") #remove alpha channel
    else:
        folder = '../images/'
        with open(folder+file, 'rb') as f:
            img = Image.open(f)
            img = img.convert('RGB')
    return img
def get_batch(size=1):
    images = []
    y = []
    idx = np.random.randint(0,len(train_data)-size)
    for i,image in enumerate(train_data[idx:idx+size]):
        img_url = image['url']
        # img = get_image(img_url, url=True)
        img = get_image(image['file_name'])
        images.append(img)
        try:
            output = id2OAOneHot[str(image['id'])]
        # print(image['id'],output)
        except:
            output = torch.zeros(len(UniqeOAPairs)) #if OA pair doesn't exist
        y.append(output)
        
    return images,np.array(y)

get_batch(1)

In [5]:
import torch
ims, y = get_batch(20)

feature_map = Backbone.Backbone()(ims) #output from backbone
position_encoded_feature_map = PositionEncoder()(feature_map)
B,C,W,H = position_encoded_feature_map.shape # (Batch, Channels, Width, Height)
backbone_output_vectorized = position_encoded_feature_map.reshape((B,C,W*H)) #changing 2d image to vector (B, C, F) (F = feature)
backbone_output_tansformed = backbone_output_vectorized.permute([0,2,1]) # (B, F, C)
# backbone_output_tansformed = position_encoded_feature_map.permute([0,3,2,1]) # (B, H,W, C)
encoder_output = Encoder()(backbone_output_tansformed) # (B, F, C)

OASampler = OA_Candidate_Sampler.OACandidateSampler(num_oas)
probabilities = OASampler(encoder_output)

print(probabilities.shape)
# confidences, indices = torch.topk(probabilities, 5, dim=-1)
# print("Top-K OA Candidates (indices):", indices)
# print("Top-K OA Candidates (confidences):", confidences)

In [ ]:
import HOI_Decoder, HOI_Prediction, SpatialFeatureGenerator, Semantic_Feature_Generator, Feature_Aggregator, Query_Refiner, HOI_Decoder, HOI_Prediction

reload(Query_Refiner)
reload(HOI_Prediction)
backbone = Backbone.Backbone()
positionEncoder = PositionEncoder()
matrix2Vector = VectorizeFeatureMap()
encoder = Encoder()
OASampler = OA_Candidate_Sampler.OACandidateSampler(num_oas)
spatial_feature_generator = SpatialFeatureGenerator.SpatialFeatureGenerator([])
semantic_generator = Semantic_Feature_Generator.SemanticFeatureGenerator()
aggregator = Feature_Aggregator.FeatureAggregator()
query_refiner = Query_Refiner.QueryRefiner(batch_size)
decoder = HOI_Decoder.HOIDecoder()
hoi_predictor = HOI_Prediction.HOIPrediction()

In [ ]:
batch_size = 2
imgs, y = get_batch(batch_size)

# Step 1: Initialize and call Backbone
features = Backbone.Backbone()(imgs)

# Step 2: Pass Backbone output to PositionEncoder
position_encoded = PositionEncoder()(features)

# Step 3: Pass PositionEncoder output to VectorizeFeatureMap
vectorized_features = VectorizeFeatureMap()(position_encoded)

# Step 4: Pass VectorizeFeatureMap output to Encoder
encoded_features = Encoder()(vectorized_features)

# Step 5: Pass Encoder output to OA_Candidate_Sampler
oas = OA_Candidate_Sampler.OACandidateSampler(num_oas)(encoded_features)

# Step 6: Pass OA_Candidate_Sampler output to SpatialFeatureGenerator
spatial_features = SpatialFeatureGenerator.SpatialFeatureGenerator([])(oas)

# Step 7: Pass SpatialFeatureGenerator output to SemanticFeatureGenerator
semantic_features = Semantic_Feature_Generator.SemanticFeatureGenerator()(oas)

# Step 8: Pass both SpatialFeatureGenerator and SemanticFeatureGenerator outputs to FeatureAggregator
aggregated_features = Feature_Aggregator.FeatureAggregator()(spatial_features, semantic_features)

# Step 9: Pass FeatureAggregator output to QueryRefiner
refined_query = Query_Refiner.QueryRefiner(batch_size)(aggregated_features)

# Step 10: Pass QueryRefiner output to HOI_Decoder
decoded_query = HOI_Decoder.HOIDecoder()(encoded_features, refined_query)

# Step 11: Pass HOI_Decoder output to HOI_Prediction
bh, bo, po, pa = HOI_Prediction.HOIPrediction()(decoded_query)

# Final Output
# print("Human-Object Interaction Predictions:", hoi_predictions)


In [2]:
import Datasets
import hico_dataset

reload(Datasets)
reload(hico_dataset)

# dataset = Datasets.VCOCODataset(img_set="train", img_folder="../images/",anno_file="annotations/trainval_vcoco.json" , num_queries=49, transforms=None)
dataset = hico_dataset.HICODetection(img_set="train", img_folder="../../HOTR/hico_20160224_det/images/train2015/",anno_file="annotations/trainval_hico.json" , num_queries=49, transforms=None)

In [ ]:
# from torch.utils.data import DataLoader, DistributedSampler
# import Set_Criterion
# import Matcher
# reload(Set_Criterion)

# batch_size = 10

# loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn= collate_fn)

# losses = ['obj_labels', 'verb_labels', 'sub_obj_boxes', 'obj_cardinality']


# weight_dict = {}
# weight_dict['loss_obj_ce'] = 1
# weight_dict['loss_verb_ce'] = 1
# weight_dict['loss_sub_bbox'] = 5
# weight_dict['loss_obj_bbox'] = 5
# weight_dict['loss_sub_giou'] = 2
# weight_dict['loss_obj_giou'] = 2

# criterion= Set_Criterion.SetCriterionHOI(num_obj_classes=20, 
#                                          num_queries=49, 
#                                          num_verb_classes=29, 
#                                          matcher=Matcher.HungarianMatcherHOI(), 
#                                          weight_dict=weight_dict, 
#                                          eos_coef= 0.1 , 
#                                          losses= losses, verb_loss_type= 'focal')

In [3]:
import Set_Criterion
import Matcher
reload(Set_Criterion)
losses = ['obj_labels', 'verb_labels', 'sub_obj_boxes', 'obj_cardinality']

# weight_dict = model.state_dict()

weight_dict = {}
weight_dict['loss_obj_ce'] = 1
weight_dict['loss_verb_ce'] = 1
weight_dict['loss_sub_bbox'] = 5
weight_dict['loss_obj_bbox'] = 5
weight_dict['loss_sub_giou'] = 2
weight_dict['loss_obj_giou'] = 2

verbs_coco = 29
n_objs_coco = 90

verbs_hico = 117
n_objs_hico = 90

criterion= Set_Criterion.SetCriterionHOI(num_obj_classes=n_objs_hico, 
                                         num_queries=49, 
                                         num_verb_classes=verbs_hico, 
                                         matcher=Matcher.HungarianMatcherHOI(), 
                                         weight_dict=weight_dict, 
                                         eos_coef= 0.1 , 
                                         losses= losses, verb_loss_type= 'focal')

In [10]:
# # model = None
# # def train_one_epoch():
# #     model.train()
# #     criterion.train()

# batch_size = 10
# loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn= collate_fn)

# import SSRT

# reload(SSRT)
# model = SSRT.SSRT(num_oas, batch_size=batch_size, nobj = 91, nact =29)


# for images, targets in loader:
#     # images = images.to(device)
#     # targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

#     outputs = model(list(images))
#     loss_dict = criterion(outputs, targets)
#     weight_dict = criterion.weight_dict
#     losses = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)

#     # reduce losses over all GPUs for logging purposes
#     # loss_dict_reduced = utils.reduce_dict(loss_dict)
#     loss_dict_reduced = loss_dict
#     loss_dict_reduced_unscaled = {f'{k}_unscaled': v
#                                     for k, v in loss_dict_reduced.items()}
#     loss_dict_reduced_scaled = {k: v * weight_dict[k]
#                                 for k, v in loss_dict_reduced.items() if k in weight_dict}
#     losses_reduced_scaled = sum(loss_dict_reduced_scaled.values())

#     # loss_value = losses_reduced_scaled.item()
#     losses.backward()

#     # return {k: meter.global_avg for k, meter in metric_logger.meters.items()}




In [4]:
# ------------------------------------------------------------------------
# Copyright (c) Hitachi, Ltd. All Rights Reserved.
# Licensed under the Apache License, Version 2.0 [see LICENSE for details]
# ------------------------------------------------------------------------
# Modified from DETR (https://github.com/facebookresearch/detr)
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved
# ------------------------------------------------------------------------
"""
Train and eval functions used in main.py
"""
import math
import os
import sys
from typing import Iterable
import numpy as np
import copy
import itertools
from torchvision import transforms
import torch

import misc 

reload(misc)


utils = misc


def train_one_epoch(model, criterion: torch.nn.Module,
                    data_loader: Iterable, optimizer: torch.optim.Optimizer,
                    device: torch.device, epoch: int, max_norm: float = 0):
    model.train()
    criterion.train()
    metric_logger = utils.MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', utils.SmoothedValue(window_size=1, fmt='{value:.6f}'))
    if hasattr(criterion, 'loss_labels'):
        metric_logger.add_meter('class_error', utils.SmoothedValue(window_size=1, fmt='{value:.2f}'))
    else:
        metric_logger.add_meter('obj_class_error', utils.SmoothedValue(window_size=1, fmt='{value:.2f}'))
    header = 'Epoch: [{}]'.format(epoch)
    print_freq = 1

    
    for samples, targets in metric_logger.log_every(data_loader, print_freq, header):
        # samples = samples.to(device)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        # targets = torch.stack(targets, dim = 0).to(device)
        # samples = list(samples)
       

        # Apply transformation to each image and convert it to tensor
        # samples = [transform(image) for image in samples]
        # samples = torch.stack(samples, dim=0)

        outputs = model(samples)
        loss_dict = criterion(outputs, targets)
        weight_dict = criterion.weight_dict
        losses = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)

        # reduce losses over all GPUs for logging purposes
        loss_dict_reduced = utils.reduce_dict(loss_dict)
        loss_dict_reduced_unscaled = {f'{k}_unscaled': v
                                      for k, v in loss_dict_reduced.items()}
        loss_dict_reduced_scaled = {k: v * weight_dict[k]
                                    for k, v in loss_dict_reduced.items() if k in weight_dict}
        losses_reduced_scaled = sum(loss_dict_reduced_scaled.values())

        loss_value = losses_reduced_scaled.item()

        if not math.isfinite(loss_value):
            print("Loss is {}, stopping training".format(loss_value))
            print(loss_dict_reduced)
            sys.exit(1)

        optimizer.zero_grad()
        losses.backward()
        if max_norm > 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)
        optimizer.step()

        metric_logger.update(loss=loss_value, **loss_dict_reduced_scaled, **loss_dict_reduced_unscaled)
        if hasattr(criterion, 'loss_labels'):
            metric_logger.update(class_error=loss_dict_reduced['class_error'])
        else:
            metric_logger.update(obj_class_error=loss_dict_reduced['obj_class_error'])
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])
    # gather the stats from all processes
    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger)
    return {k: meter.global_avg for k, meter in metric_logger.meters.items()}


In [19]:
# from torch.utils.data.dataloader import DataLoader
# def collate_fn(batch):
#     batch = list(zip(*batch))
#     return tuple(batch)


# batch_size = 10
# loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn= collate_fn)

# # nobj = n_objs_hico
# nact = verbs_hico


# nobj = 91
# # nact = verbs_coco

# import SSRT
# reload(SSRT)
# device = torch.device("cuda")
# OA_distn = load_OA_distributions(device)
# num_oas = len(OA_distn.keys())
# model = SSRT.SSRT(num_oas, OA_distn = OA_distn, batch_size=batch_size, nobj = nobj, nact =nact, device = device, n_queries = 49 )
# model.to(device)
# optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)


train_one_epoch(model=model,criterion=criterion, optimizer=optimizer, data_loader=loader,device=device, epoch=1 )

Epoch: [1]  [  0/100]  eta: 0:02:13  lr: 0.010000  obj_class_error: 100.00  loss: 11513.6934 (11513.6934)  loss_obj_ce: 3.6745 (3.6745)  loss_verb_ce: 116.8769 (116.8769)  loss_sub_bbox: 6006.5488 (6006.5488)  loss_obj_bbox: 5381.6826 (5381.6826)  loss_sub_giou: 2.8022 (2.8022)  loss_obj_giou: 2.1083 (2.1083)  loss_obj_ce_unscaled: 3.6745 (3.6745)  obj_class_error_unscaled: 100.0000 (100.0000)  loss_verb_ce_unscaled: 116.8769 (116.8769)  loss_sub_bbox_unscaled: 1201.3098 (1201.3098)  loss_obj_bbox_unscaled: 1076.3365 (1076.3365)  loss_sub_giou_unscaled: 1.4011 (1.4011)  loss_obj_giou_unscaled: 1.0541 (1.0541)  obj_cardinality_error_unscaled: 7.1000 (7.1000)  time: 1.3308  data: 0.0319  max mem: 29070
Epoch: [1]  [  1/100]  eta: 0:02:02  lr: 0.010000  obj_class_error: 100.00  loss: 11473.6367 (11493.6650)  loss_obj_ce: 3.5928 (3.6336)  loss_verb_ce: 116.8769 (189.1560)  loss_sub_bbox: 5361.7744 (5684.1616)  loss_obj_bbox: 5381.6826 (5611.9609)  loss_sub_giou: 2.1338 (2.4680)  loss_obj_g

KeyboardInterrupt: 

In [7]:
import torch
import numpy as np

def replace_nan_with_zero(arr):
    """
    Replaces any 'nan' values in a list with 0 (handles both 1D and 2D arrays).
    """
    if isinstance(arr, list):
        return [[0 if isinstance(i, float) and np.isnan(i) else i for i in sublist] for sublist in arr]
    return arr

def safely_evaluate(value):
    """
    Safely evaluates a string, replacing 'nan' with float('nan').
    """
    value = value.replace('nan', 'float("nan")')  # Replace 'nan' with float('nan')
    try:
        return eval(value)
    except Exception as e:
        print(f"Error evaluating value: {value} -> {e}")
        return None


def load_OA_distributions(device):
    data = {}

    with open('unique_means.txt', 'r') as file:
        for i, line in enumerate(file):
            # Split the line by semicolon
            parts = line.strip().split(';')

            # Extract and process each part
            item_action = parts[0].strip()
            word_list = item_action.strip("()").replace("'", "").split(", ")


            # Convert list_1, array_1, list_2, array_2 into valid Python objects using safely_evaluate
            list_1 = safely_evaluate(parts[1].strip())  # Convert string to list, e.g., [0.197, 0.297]
            array_1 = safely_evaluate(parts[2].strip())  # Convert string to nested list, e.g., [[nan, nan], [nan, nan]]
            list_2 = safely_evaluate(parts[3].strip())  # Convert string to list, e.g., [-1.82, -1.58]
            array_2 = safely_evaluate(parts[4].strip())  # Convert string to nested list, e.g., [[nan, nan], [nan, nan]]

            if list_1 is None or array_1 is None or list_2 is None or array_2 is None:
                continue  # Skip this line if evaluation failed

            # Replace 'nan' with 0 in array_1 and array_2
            array_1 = replace_nan_with_zero(array_1)
            array_2 = replace_nan_with_zero(array_2)
            

            small_value = 1e-3


            data[i] = {
                'OA_pair': word_list,
                'mean_xy': torch.tensor(list_1, dtype=torch.float32).to(device)  if isinstance(list_1, list) and all(isinstance(i, (int, float)) for i in list_1) else None,
                'cov_xy': (torch.tensor(array_1, dtype=torch.float32) + small_value * torch.eye(len(array_2), dtype=torch.float32)).to(device) if isinstance(array_1, list) and all(isinstance(i, list) and all(isinstance(j, (int, float)) for j in i) for i in array_1) else None,
                'mean_wh': torch.tensor(list_2, dtype=torch.float32).to(device) if isinstance(list_2, list) and all(isinstance(i, (int, float)) for i in list_2) else None,
                'cov_wh': (torch.tensor(array_2, dtype=torch.float32) + + small_value * torch.eye(len(array_2), dtype=torch.float32)).to(device) if isinstance(array_2, list) and all(isinstance(i, list) and all(isinstance(j, (int, float)) for j in i) for i in array_2) else None
            }


        return data
